## MNIST Fashion Dataset Maker
### Client NonIID

In [ ]:
import os
import random

import tensorflow as tf
import numpy as np
import pandas as pd
import PIL.Image as Image

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
images = np.concatenate((train_images, test_images))
labels = np.concatenate((train_labels, test_labels))

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data
labelnames = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
labelnames

In [ ]:
div = pd.read_csv('div.csv')
div.fillna(0, inplace=True)
div

In [ ]:
# Count each labels
total = {}
for idx, data in enumerate(zip(images, labels), start=0):
    label = labelnames[data[1]]
    total[label] = total.get(label, 0) + 1
total

In [ ]:
for column in div.columns[1:]:
    div = div.assign(**{column: np.around(div.loc[:,column]*total[column])})
div

In [ ]:
counter = {}
output = os.path.abspath(os.path.expanduser('dataset-mnistfashion'))
for idx, data in enumerate(zip(images, labels), start=0):
    image = Image.fromarray(data[0])
    label = labelnames[data[1]]
    num = counter.get(label, 0)
    party = random.choice(div.loc[div.loc[:, label] > 0, 'Clients'].values[:-1])
    odir = os.path.join(output, f'{party}', label)
    os.makedirs(odir, exist_ok=True)
    opath = os.path.join(odir, f'{num:04d}.jpg')
    image.save(opath)
    div.loc[div.loc[:, 'Clients'] == party, label] = div.loc[div.loc[:, 'Clients'] == party, label] - 1
    counter[label] = num + 1

In [ ]:
div